# Project Introduction 

This project explores the Kaggle Disaster Tweets dataset and aims to predict whether a tweet is about a real disaster or not. The dataset contains 10,000 tweets that were hand classified. The goal is to build a machine learning model that can predict which tweets are about a real disaster and which are not.

The project is divided into the following sections:

1. Data Preprocessing
2.  Exploration (EDA)
3. Model Building
4. Model Evaluation
5. Model Prediction on Test Data
6. Conclusion

In [2]:
# Import all the necessary libraries for NLP and data processing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Data Preparation and Preprocessing

The first step is to load the data and perform some basic data preprocessing. The data is loaded into a pandas dataframe and the first few rows are displayed to get an idea of the data. The data contains the following columns:

1. id: A unique identifier for each tweet
2. keyword: A keyword from the tweet
3. location: The location the tweet was sent from
4. text: The text of the tweet
5. target: Whether the tweet is about a real disaster (1) or not (0)

In [3]:
tweets = pd.read_csv("./nlp-getting-started/train.csv")
tweets.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
# Check for missing values
print(tweets.isnull().sum())

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64
